<a href="https://colab.research.google.com/github/kiaerii/mash/blob/main/2lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Функция парсинга новостей
def parse_news(news_count=10):
    url = "https://lenta.ru/"

    try:
        response = requests.get(url) # Отправляем запрос к сайту
        soup = BeautifulSoup(response.content, 'html.parser') # Создаем объект для парсинга HTML
        news_list = []

        #Ищем все новостные блоки
        news_items = soup.find_all('a', class_=lambda x: x and ('card' in x or 'toptitle' in x))

        #Перебираем и смотрим кол-во
        for item in news_items:
            if len(news_list) >= news_count:
                break

            try:
                title_elem = item.find(['h3', 'span'])  # ищем в тегах h3 или span (заголовки новостей в своей HTML-структуре)
                if title_elem:
                    title = title_elem.get_text(strip=True)  # извлекаем текст без пробелов

                    # получаем ссылку на новость
                    link = item.get('href')  # получаем атрибут href (Он задаёт URL-адрес страницы или документа)
                    if link and not link.startswith('http'):
                        link = 'https://lenta.ru' + link

                    #парсим и добавляем данные новости в список
                    full_text = parse_full_text(link) if link else ""
                    news_list.append({
                        'title': title,
                        'link': link,
                        'full_text': full_text
                    })

            except Exception as e:
                continue

        return pd.DataFrame(news_list)

    except Exception as e:
        print(f"Ошибка: {e}")
        return pd.DataFrame()

# Функция для парсинга полного текста новости
def parse_full_text(url):
    # Парсим полный текст новости по ссылке
    try:
        response = requests.get(url, timeout=5) # Отправляем запрос к странице новости с таймингом 5 секунд
        soup = BeautifulSoup(response.content, 'html.parser')

        # Ищем текстовые блоки новости
        text_blocks = soup.find_all('p', class_=lambda x: x and 'body' in x)  # ищем параграфы с классом содержащим 'body'
        if not text_blocks:
            text_blocks = soup.find_all('div', class_=lambda x: x and 'body' in x)

        # Объединяем текст из всех блоков в одну строку
        full_text = ' '.join([block.get_text(strip=True) for block in text_blocks])
        return full_text[:500]  # возвращаем первые 500 символов текста

    except:  # если любая ошибка при парсинге текста
        return ""

# Основная программа
if __name__ == "__main__":
    # Ввод количества новостей с клавиатуры
    while True:
        try:
            count = int(input("Введите количество новостей для парсинга: "))
            if count > 0:
                break
            else:
                print("Число должно быть больше 0!")
        except ValueError:
            print("Пожалуйста, введите целое число!")

    df = parse_news(count)
    print(f"\nУспешно собрано {len(df)} новостей:")

    # Перебираем все новости в DataFrame
    for i, row in df.iterrows():  # iterrows() возвращает индекс и строку
        print(f"{i+1}. {row['title']}")  #выводим номер и заголовок новости
        #Если есть полный текст новости
        if row['full_text']:
            print(f"   Текст: {row['full_text'][:100]}...")  # выводим первые 100 символов
        print()  # пустая строка для разделения

    df.to_csv('news.csv', index=False, encoding='utf-8')  # сохраняем в csv без индексов в UTF-8

Введите количество новостей для парсинга: 6

Успешно собрано 6 новостей:
1. Отказ от НАТО, признание Крыма, Донецка и Луганска российскими. О чем говорится в мирном плане США по Украине?
   Текст: Нардеп Верховной рады Алексей Гончаренко (внесен в перечень террористов и экстремистов Росфинмонитор...

2. В МИД России прокомментировали освобождение Купянска
   Текст: Освобождение российскими бойцами Купянска позволит начать сбор доказательств о военных преступлениях...

3. Тренер Нурмагомедова и Махачева ответил на вопрос о сравнении двух бойцов
   Текст: Хавьер Мендес, тренерХабиба НурмагомедоваиИслама Махачева, ответил на вопрос о сравнении двух россий...

4. В Белом доме выразили уверенность в остановке конфликта на Украине
   Текст: Военный конфликт наУкраинебудет решен, несмотря на наличие сомнений в этом. Об этом заявилапресс-сек...

5. Зеленский отказался увольнять Ермака
   Текст: Украинский лидерВладимир Зеленскийотказался отправить в отставку главу своего офисаАндрея Ермакана ф